# <b>Snow Pack Prediction Challenge</b>

## <b> Objective </b> 
Develop and implement a model to predict daily Snow Water Equivalent (SWE) for the winter season (December 1 to May 31) across the Western United States using spatio-temporal data.

### Parsing the data

#### SWE Data

In [40]:
import pandas as pd

# parse swe_data
station_info = pd.read_csv('./swe_data/Station_Info.csv')
swe_values = pd.read_csv('./swe_data/SWE_values_all.csv')

In [42]:
station_info

,Station,Latitude,Longitude,Elevation,Southness
0,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565
1,Bateman,36.51174,-106.31543,9249,-0.465294
2,Battle Mountain,41.05402,-107.26674,7498,-0.812550
3,Bear Trap Meadow,43.88743,-107.06135,8200,-0.747935
4,Beaver Creek,44.94966,-111.35852,7850,0.021185
...,...,...,...,...,...
195,Widtsoe 3,37.83633,-111.88163,9640,-0.063832
196,Willow Creek Pass,40.34734,-106.09520,9523,-0.642395
197,Wolf Creek Summit,37.47903,-106.80234,10957,0.286872
198,Wood Creek,47.44847,-112.81428,5960,0.790996


In [44]:
swe_values

,Date,SWE,Latitude,Longitude
0,1991-01-01,111.76,45.19085,-119.25392
1,1991-01-02,111.76,45.19085,-119.25392
2,1991-01-03,111.76,45.19085,-119.25392
3,1991-01-04,111.76,45.19085,-119.25392
4,1991-01-05,111.76,45.19085,-119.25392
...,...,...,...,...
1899395,2016-12-27,30.48,33.81259,-110.91852
1899396,2016-12-28,30.48,33.81259,-110.91852
1899397,2016-12-29,30.48,33.81259,-110.91852
1899398,2016-12-30,27.94,33.81259,-110.91852


#### Meteorological Data

In [46]:
#parse meteorological_data
md_precip = pd.read_csv('./meteorological_data/Modified_Output_precip.csv')
md_Rmax = pd.read_csv('./meteorological_data/Modified_Output_Rmax.csv')
md_Rmin = pd.read_csv('./meteorological_data/Modified_Output_Rmin.csv')
md_SPH = pd.read_csv('./meteorological_data/Modified_Output_SPH.csv')
md_SRAD = pd.read_csv('./meteorological_data/Modified_Output_SRAD.csv')
md_tmax = pd.read_csv('./meteorological_data/Modified_Output_tmax.csv')
md_tmin = pd.read_csv('./meteorological_data/Modified_Output_tmin.csv')
md_windspeed = pd.read_csv('./meteorological_data/Modified_Output_windspeed.csv')

#### Additional Test Location Data

In [48]:
# parse addtional_test_locations
atl_static = pd.read_csv('./additional_test_locations/Test_InputData_staticVars_2017_2019.csv')
atl_dynamic = pd.read_csv('./additional_test_locations/Test_InputData_dynamicVars_2017_2019.csv')

### Cleaning the data

#### SWE Data

In [50]:
# merge station_info and swe_values
merged_swe = pd.merge(station_info, swe_values, on=['Latitude', 'Longitude'], how='left')

In [52]:
merged_swe

,Station,Latitude,Longitude,Elevation,Southness,Date,SWE
0,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-01,111.76
1,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-02,111.76
2,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-03,111.76
3,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-04,111.76
4,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-05,111.76
...,...,...,...,...,...,...,...
1899395,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-27,30.48
1899396,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-28,30.48
1899397,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-29,30.48
1899398,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-30,27.94


#### Meteorological Data

In [54]:
# rename individual meteorological data columns appropiately
md_precip = md_precip.rename(columns={'variable_value':'precip'})
md_Rmax = md_Rmax.rename(columns={'variable_value':'Rmax'})
md_Rmin = md_Rmin.rename(columns={'variable_value':'Rmin'})
md_SPH = md_SPH.rename(columns={'variable_value':'SPH'})
md_SRAD = md_SRAD.rename(columns={'variable_value':'SRAD'})
md_tmax = md_tmax.rename(columns={'variable_value':'tmax'})
md_tmin = md_tmin.rename(columns={'variable_value':'tmin'})
md_windspeed = md_windspeed.rename(columns={'variable_value':'windspeed'})

In [56]:
# merge all individual meteorological data into one dataframe
merged_md = pd.merge(md_precip, md_Rmax, on=['date', 'lat', 'lon'], how='inner')
merged_md = pd.merge(merged_md, md_Rmin, on=['date', 'lat', 'lon'], how='inner')
merged_md = pd.merge(merged_md, md_SPH, on=['date', 'lat', 'lon'], how='inner')
merged_md = pd.merge(merged_md, md_SRAD, on=['date', 'lat', 'lon'], how='inner')
merged_md = pd.merge(merged_md, md_tmax, on=['date', 'lat', 'lon'], how='inner')
merged_md = pd.merge(merged_md, md_tmin, on=['date', 'lat', 'lon'], how='inner')
merged_md = pd.merge(merged_md, md_windspeed, on=['date', 'lat', 'lon'], how='inner')

In [58]:
merged_md

,date,lat,lon,precip,Rmax,Rmin,SPH,SRAD,tmax,tmin,windspeed
0,1990-01-02,43.40625,-109.03125,0.000,83.81,33.77,0.0020,75.800,2.83,-10.28,3.83
1,1990-01-03,43.40625,-109.03125,0.000,79.54,29.78,0.0011,83.150,-2.86,-15.86,3.16
2,1990-01-05,43.40625,-109.03125,0.000,79.02,38.86,0.0015,82.175,0.79,-13.11,3.46
3,1990-01-07,43.40625,-109.03125,0.000,81.46,34.56,0.0025,79.125,6.31,-6.86,4.53
4,1990-01-09,43.40625,-109.03125,0.000,81.02,33.54,0.0028,79.200,8.69,-5.16,4.93
...,...,...,...,...,...,...,...,...,...,...,...
3631115,2019-12-19,48.78125,-121.71875,37.325,100.00,83.70,0.0045,37.450,3.61,-5.13,6.16
3631116,2019-12-25,48.78125,-121.71875,0.000,98.00,65.63,0.0027,58.500,-0.01,-7.33,2.47
3631117,2019-12-26,48.78125,-121.71875,3.200,97.80,71.07,0.0026,57.825,-1.33,-7.34,3.83
3631118,2019-12-27,48.78125,-121.71875,4.025,100.00,76.37,0.0035,40.125,1.30,-6.58,3.37


In [73]:
# rename Longitude and latitude column
merged_md = merged_md.rename(columns={'date':'Date', 'lat':'Latitude', 'lon':'Longitude'})
merged_md

,Date,Latitude,Longitude,precip,Rmax,Rmin,SPH,SRAD,tmax,tmin,windspeed
0,1990-01-02,43.40625,-109.03125,0.000,83.81,33.77,0.0020,75.800,2.83,-10.28,3.83
1,1990-01-03,43.40625,-109.03125,0.000,79.54,29.78,0.0011,83.150,-2.86,-15.86,3.16
2,1990-01-05,43.40625,-109.03125,0.000,79.02,38.86,0.0015,82.175,0.79,-13.11,3.46
3,1990-01-07,43.40625,-109.03125,0.000,81.46,34.56,0.0025,79.125,6.31,-6.86,4.53
4,1990-01-09,43.40625,-109.03125,0.000,81.02,33.54,0.0028,79.200,8.69,-5.16,4.93
...,...,...,...,...,...,...,...,...,...,...,...
3631115,2019-12-19,48.78125,-121.71875,37.325,100.00,83.70,0.0045,37.450,3.61,-5.13,6.16
3631116,2019-12-25,48.78125,-121.71875,0.000,98.00,65.63,0.0027,58.500,-0.01,-7.33,2.47
3631117,2019-12-26,48.78125,-121.71875,3.200,97.80,71.07,0.0026,57.825,-1.33,-7.34,3.83
3631118,2019-12-27,48.78125,-121.71875,4.025,100.00,76.37,0.0035,40.125,1.30,-6.58,3.37


#### Additional Test Location Data

In [64]:
# merge addtional_test_locations static and dynamic data
merged_atl = pd.merge(atl_static, atl_dynamic, on=['lat', 'lon'], how='left')

In [66]:
merged_atl

,lat,lon,Elevation,Southness,date,precip,Rmax,Rmin,SPH,SRAD,tmax,tmin,windspeed
0,45.21875,-119.21875,5770,-0.124565,2017-01-01,3.575,100.00,71.78,0.0024,54.700,-3.09,-9.90,4.56
1,45.21875,-119.21875,5770,-0.124565,2017-01-02,0.600,99.67,67.66,0.0020,28.300,-5.27,-11.50,1.96
2,45.21875,-119.21875,5770,-0.124565,2017-01-03,0.000,82.28,55.65,0.0013,50.275,-7.88,-13.37,3.40
3,45.21875,-119.21875,5770,-0.124565,2017-01-04,0.000,96.78,44.43,0.0010,72.725,-9.18,-19.19,5.92
4,45.21875,-119.21875,5770,-0.124565,2017-01-05,0.000,81.10,27.47,0.0008,84.150,-5.78,-18.93,3.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10945,39.28125,-117.09375,8685,-0.168625,2019-12-27,0.000,67.91,54.06,0.0016,104.400,-6.22,-10.46,2.61
10946,39.28125,-117.09375,8685,-0.168625,2019-12-28,0.000,66.72,48.63,0.0016,104.225,-5.12,-10.48,3.26
10947,39.28125,-117.09375,8685,-0.168625,2019-12-29,1.650,66.96,41.88,0.0021,87.675,0.73,-6.87,2.23
10948,39.28125,-117.09375,8685,-0.168625,2019-12-30,0.900,74.63,60.05,0.0026,53.175,-1.20,-5.47,1.89


#### Create the Final Data

To get the final data, we have to combine the swe data and the meteorological data. The latitudes and longitudes do not match directly, therefore we will use spatial coordinates and the nearest neigbors to determine data close to merge data close to the stations

In [71]:
from scipy.spatial import cKDTree

# extract spatial coordinates
swe_coords = merged_swe[['Latitude', 'Longitude']].values
md_coords = merged_md[['Latitude', 'Longitude']].drop_duplicates().values

# build KD-Tree using meteorological grid points
tree = cKDTree(md_coords)

# find the nearest meteorological grid point for each SWE station
distances, indices = tree.query(swe_coords, k=1)

# map each station to its closest meteorological grid point
merged_swe['nearest_md_latitude'] = md_coords[indices, 0]
merged_swe['nearest_md_longitude'] = md_coords[indices, 1]


In [75]:
merged_swe

,Station,Latitude,Longitude,Elevation,Southness,Date,SWE,nearest_md_latitude,nearest_md_longitude
0,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-01,111.76,45.21875,-119.21875
1,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-02,111.76,45.21875,-119.21875
2,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-03,111.76,45.21875,-119.21875
3,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-04,111.76,45.21875,-119.21875
4,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-05,111.76,45.21875,-119.21875
...,...,...,...,...,...,...,...,...,...
1899395,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-27,30.48,33.78125,-110.90625
1899396,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-28,30.48,33.78125,-110.90625
1899397,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-29,30.48,33.78125,-110.90625
1899398,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-30,27.94,33.78125,-110.90625


In [77]:
merged_md

,Date,Latitude,Longitude,precip,Rmax,Rmin,SPH,SRAD,tmax,tmin,windspeed
0,1990-01-02,43.40625,-109.03125,0.000,83.81,33.77,0.0020,75.800,2.83,-10.28,3.83
1,1990-01-03,43.40625,-109.03125,0.000,79.54,29.78,0.0011,83.150,-2.86,-15.86,3.16
2,1990-01-05,43.40625,-109.03125,0.000,79.02,38.86,0.0015,82.175,0.79,-13.11,3.46
3,1990-01-07,43.40625,-109.03125,0.000,81.46,34.56,0.0025,79.125,6.31,-6.86,4.53
4,1990-01-09,43.40625,-109.03125,0.000,81.02,33.54,0.0028,79.200,8.69,-5.16,4.93
...,...,...,...,...,...,...,...,...,...,...,...
3631115,2019-12-19,48.78125,-121.71875,37.325,100.00,83.70,0.0045,37.450,3.61,-5.13,6.16
3631116,2019-12-25,48.78125,-121.71875,0.000,98.00,65.63,0.0027,58.500,-0.01,-7.33,2.47
3631117,2019-12-26,48.78125,-121.71875,3.200,97.80,71.07,0.0026,57.825,-1.33,-7.34,3.83
3631118,2019-12-27,48.78125,-121.71875,4.025,100.00,76.37,0.0035,40.125,1.30,-6.58,3.37


In [79]:
# rename for clarity before merging
merged_md = merged_md.rename(columns={'Latitude': 'nearest_md_latitude', 'Longitude': 'nearest_md_longitude'})

# merge meteorological data with swe data using the nearest matched latitude and longitude
final_data = pd.merge(
    merged_swe, merged_md,
    on=['Date', 'nearest_md_latitude', 'nearest_md_longitude'],
    how='left'
)

In [83]:
final_data

,Station,Latitude,Longitude,Elevation,Southness,Date,SWE,nearest_md_latitude,nearest_md_longitude,precip,Rmax,Rmin,SPH,SRAD,tmax,tmin,windspeed
0,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-01,111.76,45.21875,-119.21875,0.00,92.91,64.75,0.0025,71.250,-0.86,-7.94,4.01
1,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-02,111.76,45.21875,-119.21875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-03,111.76,45.21875,-119.21875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-04,111.76,45.21875,-119.21875,0.00,99.88,62.76,0.0014,77.900,-6.28,-15.51,2.90
4,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-05,111.76,45.21875,-119.21875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899395,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-27,30.48,33.78125,-110.90625,0.00,50.49,17.52,0.0026,137.450,15.31,3.37,3.40
1899396,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-28,30.48,33.78125,-110.90625,0.00,45.33,16.91,0.0026,128.700,15.99,4.20,3.45
1899397,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-29,30.48,33.78125,-110.90625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1899398,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-30,27.94,33.78125,-110.90625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
# fill missing data
final_data.ffill(inplace=True) #using forward fill because it is a time series prediction

In [87]:
final_data

,Station,Latitude,Longitude,Elevation,Southness,Date,SWE,nearest_md_latitude,nearest_md_longitude,precip,Rmax,Rmin,SPH,SRAD,tmax,tmin,windspeed
0,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-01,111.76,45.21875,-119.21875,0.00,92.91,64.75,0.0025,71.250,-0.86,-7.94,4.01
1,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-02,111.76,45.21875,-119.21875,0.00,92.91,64.75,0.0025,71.250,-0.86,-7.94,4.01
2,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-03,111.76,45.21875,-119.21875,0.00,92.91,64.75,0.0025,71.250,-0.86,-7.94,4.01
3,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-04,111.76,45.21875,-119.21875,0.00,99.88,62.76,0.0014,77.900,-6.28,-15.51,2.90
4,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-05,111.76,45.21875,-119.21875,0.00,99.88,62.76,0.0014,77.900,-6.28,-15.51,2.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899395,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-27,30.48,33.78125,-110.90625,0.00,50.49,17.52,0.0026,137.450,15.31,3.37,3.40
1899396,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-28,30.48,33.78125,-110.90625,0.00,45.33,16.91,0.0026,128.700,15.99,4.20,3.45
1899397,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-29,30.48,33.78125,-110.90625,0.00,45.33,16.91,0.0026,128.700,15.99,4.20,3.45
1899398,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-30,27.94,33.78125,-110.90625,0.00,45.33,16.91,0.0026,128.700,15.99,4.20,3.45
